## 📦 BƯỚC 0: Cài Đặt và Import Thư Viện

In [4]:
# Cài đặt thư viện cần thiết
import subprocess
import sys

def install_if_needed(package):
    try:
        __import__(package.replace('-', '_'))
        print(f'✅ {package} đã cài đặt')
    except ImportError:
        print(f'📥 Đang cài đặt {package}...')
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package, '-q'])
        print(f'✅ {package} cài đặt xong')

# Cài đặt các package cần thiết
packages = ['biopython', 'lightgbm', 'scikit-learn', 'pandas', 'numpy', 'tqdm']
for pkg in packages:
    install_if_needed(pkg)

print('\n✅ Tất cả thư viện sẵn sàng!')

📥 Đang cài đặt biopython...
✅ biopython cài đặt xong
✅ lightgbm đã cài đặt
📥 Đang cài đặt scikit-learn...
✅ scikit-learn cài đặt xong
✅ pandas đã cài đặt
✅ numpy đã cài đặt
✅ tqdm đã cài đặt

✅ Tất cả thư viện sẵn sàng!


In [5]:
# Import thư viện
import pandas as pd
import numpy as np
import os
import pickle
import time
import warnings
from pathlib import Path
from collections import Counter
from tqdm import tqdm

# BioPython
from Bio import SeqIO

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import hamming_loss

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

warnings.filterwarnings('ignore')

# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('✅ Tất cả thư viện đã được import!')
print(f'📊 NumPy: {np.__version__} | Pandas: {pd.__version__}')
print(f'🔥 PyTorch: {torch.__version__} | Device: {device}')

✅ Tất cả thư viện đã được import!
📊 NumPy: 1.26.4 | Pandas: 2.2.3
🔥 PyTorch: 2.6.0+cu124 | Device: cuda


## 📁 BƯỚC 1: Thiết Lập Đường Dẫn

In [ ]:
if os.path.exists('/kaggle/input'):
    print('🔍 Đang chạy trên Kaggle')
    
    input_base = Path('/kaggle/input')
    print(f'   Available inputs: {os.listdir(input_base)}')
    
    # Find CAFA dataset
    cafa_dir = None
    for item in os.listdir(input_base):
        if 'cafa' in item.lower():
            cafa_dir = input_base / item
            break
    
    if cafa_dir is None:
        cafa_dir = input_base / os.listdir(input_base)[0]
    
    print(f'   Dataset dir: {cafa_dir}')
    print(f'   Contents: {os.listdir(cafa_dir)}')
    
    if (cafa_dir / 'Train').exists():
        TRAIN_PATH = cafa_dir / 'Train'
        TEST_PATH = cafa_dir / 'Test'
    else:
        TRAIN_PATH = cafa_dir
        TEST_PATH = cafa_dir
    
    IA_PATH = cafa_dir / 'IA.tsv'
    OUTPUT_PATH = Path('/kaggle/working')
    DATA_PATH = OUTPUT_PATH / 'data'
    MODEL_PATH = OUTPUT_PATH / 'models'
    SUBMISSION_PATH = OUTPUT_PATH  
    
else:
    print('🔍 Đang chạy Local')
    TRAIN_PATH = Path('../Train')
    TEST_PATH = Path('../Test')
    IA_PATH = Path('../IA.tsv')
    
    DATA_PATH = Path('../data/processed')
    MODEL_PATH = Path('../models')
    SUBMISSION_PATH = Path('../submissions')

for path in [DATA_PATH, MODEL_PATH, SUBMISSION_PATH]:
    path.mkdir(parents=True, exist_ok=True)

print(f'\n📁 Train: {TRAIN_PATH}')
print(f'📁 Test: {TEST_PATH}')

print(f'\n📋 File verification:')
for fname in ['train_terms.tsv', 'train_sequences.fasta']:
    fpath = TRAIN_PATH / fname
    exists = fpath.exists()
    print(f'   {fname}: {"✅" if exists else "❌"} {fpath}')
    
    if not exists:
        # Search for file
        print(f'   🔍 Searching for {fname}...')
        for root, dirs, files in os.walk('/kaggle/input'):
            if fname in files:
                print(f'      Found: {root}/{fname}')
                if 'train_terms' in fname:
                    TRAIN_PATH = Path(root)

print('\n✅ Đường dẫn đã thiết lập!')

🔍 Đang chạy trên Kaggle
   Available inputs: ['cafa-6-protein-function-prediction']
   Dataset dir: /kaggle/input/cafa-6-protein-function-prediction
   Contents: ['sample_submission.tsv', 'IA.tsv', 'Test', 'Train']

📁 Train: /kaggle/input/cafa-6-protein-function-prediction/Train
📁 Test: /kaggle/input/cafa-6-protein-function-prediction/Test

📋 File verification:
   train_terms.tsv: ✅ /kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv
   train_sequences.fasta: ✅ /kaggle/input/cafa-6-protein-function-prediction/Train/train_sequences.fasta

✅ Đường dẫn đã thiết lập!


## 📥 BƯỚC 2: Tải Dữ Liệu

In [ ]:
print('='*60)
print('📂 LOADING DATA')
print('='*60)

def extract_protein_id(fasta_id):
    """
    Extract UniProt accession từ FASTA ID
    Handles multiple formats:
    - "sp|A0A0C5B5G6|MOTSC_HUMAN" -> "A0A0C5B5G6"
    - "A0A0C5B5G6 9606" -> "A0A0C5B5G6"
    - "A0A0C5B5G6" -> "A0A0C5B5G6"
    """
    fasta_id = fasta_id.split()[0]
    
    if '|' in fasta_id:
        parts = fasta_id.split('|')
        return parts[1] if len(parts) >= 2 else fasta_id
    
    return fasta_id

# 1. Load annotations (train_terms.tsv)
print('📋 Loading train_terms.tsv...')
train_terms = pd.read_csv(TRAIN_PATH / 'train_terms.tsv', sep='\t')
print(f'   ✅ {len(train_terms):,} annotations')
print(f'   Columns: {list(train_terms.columns)}')

# Standardize column names
column_mapping = {}
if 'EntryID' in train_terms.columns:
    column_mapping['EntryID'] = 'protein_id'
if 'term' in train_terms.columns:
    column_mapping['term'] = 'go_term'

if column_mapping:
    train_terms = train_terms.rename(columns=column_mapping)
    print(f'   📝 Renamed columns: {column_mapping}')
    print(f'   New columns: {list(train_terms.columns)}')

# 2. Load protein sequences
print('\n🧬 Loading train_sequences.fasta...')
train_sequences = {}
for record in SeqIO.parse(TRAIN_PATH / 'train_sequences.fasta', 'fasta'):
    accession = extract_protein_id(record.id)
    train_sequences[accession] = str(record.seq)
print(f'   ✅ {len(train_sequences):,} sequences')

# Debug: Show first few IDs
print(f'   Sample FASTA IDs: {list(train_sequences.keys())[:3]}')
print(f'   Sample train_terms IDs: {train_terms["protein_id"].head(3).tolist()}')

# 3. Load test sequences
print('\n🧪 Loading testsuperset.fasta...')
test_sequences = {}
for record in SeqIO.parse(TEST_PATH / 'testsuperset.fasta', 'fasta'):
    accession = extract_protein_id(record.id)
    test_sequences[accession] = str(record.seq)
print(f'   ✅ {len(test_sequences):,} test sequences')

# 4. Load IA weights (nếu có)
print('\n⚖️ Loading IA.tsv...')
try:
    ia_weights = pd.read_csv(IA_PATH, sep='\t', header=None, names=['go_term', 'ia_weight'])
    print(f'   ✅ {len(ia_weights):,} GO term weights')
except:
    ia_weights = None
    print('   ⚠️ IA.tsv not found')

# Verify overlap
train_ids = set(train_terms['protein_id'].unique())
seq_ids = set(train_sequences.keys())
overlap = train_ids & seq_ids
print(f'\n🔍 ID Matching Check:')
print(f'   train_terms IDs: {len(train_ids):,}')
print(f'   train_sequences IDs: {len(seq_ids):,}')
print(f'   Overlapping IDs: {len(overlap):,}')

if len(overlap) == 0:
    print('   ⚠️ WARNING: No matching IDs! Check ID format.')
    print(f'\n   Debug - train_terms samples: {list(train_ids)[:3]}')
    print(f'   Debug - sequence samples: {list(seq_ids)[:3]}')
elif len(overlap) < len(train_ids):
    print(f'   ⚠️ Missing sequences for {len(train_ids) - len(overlap):,} proteins')
else:
    print('   ✅ All proteins have sequences!')

print('\n' + '='*60)
print('✅ DATA LOADED SUCCESSFULLY!')
print('='*60)

📂 LOADING DATA
📋 Loading train_terms.tsv...
   ✅ 537,027 annotations
   Columns: ['EntryID', 'term', 'aspect']
   📝 Renamed columns: {'EntryID': 'protein_id', 'term': 'go_term'}
   New columns: ['protein_id', 'go_term', 'aspect']

🧬 Loading train_sequences.fasta...
   ✅ 82,404 sequences
   Sample FASTA IDs: ['A0A0C5B5G6', 'A0JNW5', 'A0JP26']
   Sample train_terms IDs: ['Q5W0B1', 'Q5W0B1', 'Q5W0B1']

🧪 Loading testsuperset.fasta...
   ✅ 224,309 test sequences

⚖️ Loading IA.tsv...
   ✅ 40,122 GO term weights

🔍 ID Matching Check:
   train_terms IDs: 82,404
   train_sequences IDs: 82,404
   Overlapping IDs: 82,404
   ✅ All proteins have sequences!

✅ DATA LOADED SUCCESSFULLY!


In [ ]:
#  LOAD GO HIERARCHY (go-basic.obo)
print('='*60)
print('🌳 LOADING GO HIERARCHY')
print('='*60)

def parse_obo_file(obo_path):
    """Parse GO OBO file to extract hierarchy"""
    go_parents = {}  
    go_names = {}    
    go_namespace = {} 
    obsolete_terms = set()
    
    current_term = None
    current_name = None
    current_namespace = None
    current_parents = []
    is_obsolete = False
    
    with open(obo_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            
            if line == '[Term]':
                # Save previous term
                if current_term and not is_obsolete:
                    go_parents[current_term] = current_parents
                    go_names[current_term] = current_name
                    go_namespace[current_term] = current_namespace
                elif current_term and is_obsolete:
                    obsolete_terms.add(current_term)
                
                # Reset
                current_term = None
                current_name = None
                current_namespace = None
                current_parents = []
                is_obsolete = False
                
            elif line.startswith('id: GO:'):
                current_term = line.split('id: ')[1].strip()
                
            elif line.startswith('name: '):
                current_name = line.split('name: ')[1].strip()
                
            elif line.startswith('namespace: '):
                current_namespace = line.split('namespace: ')[1].strip()
                
            elif line.startswith('is_a: GO:'):
                parent = line.split('is_a: ')[1].split(' !')[0].strip()
                current_parents.append(parent)
                
            elif line.startswith('relationship: part_of GO:'):
                parent = line.split('part_of ')[1].split(' !')[0].strip()
                current_parents.append(parent)
                
            elif line == 'is_obsolete: true':
                is_obsolete = True
        
        # Save last term
        if current_term and not is_obsolete:
            go_parents[current_term] = current_parents
            go_names[current_term] = current_name
            go_namespace[current_term] = current_namespace
    
    return go_parents, go_names, go_namespace, obsolete_terms

def get_all_ancestors(term, go_parents, cache=None):
    """Get all ancestors of a GO term (including itself)"""
    if cache is None:
        cache = {}
    
    if term in cache:
        return cache[term]
    
    ancestors = {term}
    if term in go_parents:
        for parent in go_parents[term]:
            ancestors.update(get_all_ancestors(parent, go_parents, cache))
    
    cache[term] = ancestors
    return ancestors

def build_ancestor_cache(go_terms, go_parents):
    """Build ancestor cache for all GO terms"""
    cache = {}
    for term in tqdm(go_terms, desc='Building GO hierarchy'):
        get_all_ancestors(term, go_parents, cache)
    return cache

# Load GO hierarchy
obo_path = TRAIN_PATH / 'go-basic.obo'
if obo_path.exists():
    print(f'📂 Loading: {obo_path}')
    go_parents, go_names, go_namespace, obsolete_terms = parse_obo_file(obo_path)
    
    print(f'   ✅ GO terms: {len(go_parents):,}')
    print(f'   ✅ Obsolete terms: {len(obsolete_terms):,}')
    
    # Count by namespace
    ns_counts = {}
    for term, ns in go_namespace.items():
        ns_counts[ns] = ns_counts.get(ns, 0) + 1
    print(f'   📊 By namespace:')
    for ns, count in ns_counts.items():
        print(f'      • {ns}: {count:,}')
    
    # Build ancestor cache
    print('\n🔗 Building ancestor cache...')
    all_training_terms = set(train_terms['go_term'].unique())
    ancestor_cache = build_ancestor_cache(all_training_terms, go_parents)
    
    # Count average ancestors
    avg_ancestors = np.mean([len(ancestor_cache.get(t, {t})) for t in all_training_terms])
    print(f'   ✅ Average ancestors per term: {avg_ancestors:.1f}')
    
    GO_HIERARCHY_LOADED = True
else:
    print(f'⚠️ go-basic.obo not found at {obo_path}')
    GO_HIERARCHY_LOADED = False
    go_parents = {}
    ancestor_cache = {}

print('\n' + '='*60)

🌳 LOADING GO HIERARCHY
📂 Loading: /kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo
   ✅ GO terms: 40,122
   ✅ Obsolete terms: 7,979
   📊 By namespace:
      • biological_process: 25,950
      • molecular_function: 10,131
      • cellular_component: 4,040
      • external: 1

🔗 Building ancestor cache...


Building GO hierarchy: 100%|██████████| 26125/26125 [00:00<00:00, 240142.78it/s]

   ✅ Average ancestors per term: 13.3



## 📊 BƯỚC 3: Phân Tích Dữ Liệu

In [9]:
print('='*60)
print('📊 DATA ANALYSIS')
print('='*60)

# Thống kê cơ bản
n_proteins = train_terms['protein_id'].nunique()
n_go_terms = train_terms['go_term'].nunique()
n_annotations = len(train_terms)

print(f'\n🧬 Proteins: {n_proteins:,}')
print(f'📚 Unique GO Terms: {n_go_terms:,}')
print(f'📋 Total Annotations: {n_annotations:,}')
print(f'📈 Avg GO terms per protein: {n_annotations / n_proteins:.2f}')

# Phân bố GO terms
go_counts = train_terms['go_term'].value_counts()
print(f'\n📊 GO Term Distribution:')
print(f'   • Rare (1-10 proteins): {(go_counts <= 10).sum():,} terms')
print(f'   • Medium (11-100): {((go_counts > 10) & (go_counts <= 100)).sum():,} terms')
print(f'   • Common (>100): {(go_counts > 100).sum():,} terms')
print(f'   • Most common: {go_counts.index[0]} ({go_counts.iloc[0]:,} proteins)')

# Sequence lengths
if train_sequences:
    seq_lengths = [len(seq) for seq in train_sequences.values()]
    print(f'\n🧵 Sequence Length Stats:')
    print(f'   • Mean: {np.mean(seq_lengths):.0f}')
    print(f'   • Median: {np.median(seq_lengths):.0f}')
    print(f'   • Min/Max: {min(seq_lengths)} / {max(seq_lengths)}')

📊 DATA ANALYSIS

🧬 Proteins: 82,404
📚 Unique GO Terms: 26,125
📋 Total Annotations: 537,027
📈 Avg GO terms per protein: 6.52

📊 GO Term Distribution:
   • Rare (1-10 proteins): 18,868 terms
   • Medium (11-100): 6,594 terms
   • Common (>100): 663 terms
   • Most common: GO:0005515 (33,713 proteins)

🧵 Sequence Length Stats:
   • Mean: 526
   • Median: 409
   • Min/Max: 3 / 35213


## ⚙️ BƯỚC 4: Feature Engineering

Tạo các đặc trưng từ chuỗi protein:
1. **Amino Acid Composition (AAC)**: Tỷ lệ 20 amino acids
2. **Dipeptide Composition (DPC)**: Tỷ lệ 400 dipeptides
3. **Physicochemical Properties**: Các tính chất vật lý/hóa học
4. **Sequence Statistics**: Chiều dài, entropy, etc.

In [ ]:
print('='*60)
print('⚙️ FEATURE ENGINEERING (ENHANCED)')
print('='*60)

# Amino acids
AMINO_ACIDS = 'ACDEFGHIKLMNPQRSTVWY'
AA_TO_IDX = {aa: i for i, aa in enumerate(AMINO_ACIDS)}

# Physicochemical properties
HYDROPHOBIC = set('AILMFVWP')
HYDROPHILIC = set('RKDENQ')
POLAR = set('STNQ')
CHARGED_POS = set('RKH')
CHARGED_NEG = set('DE')
AROMATIC = set('FYW')
SMALL = set('AGSTC')
ALIPHATIC = set('ILV')
SULFUR = set('CM')
HYDROXYL = set('ST')

# Molecular weights
AA_MW = {'A': 89, 'C': 121, 'D': 133, 'E': 147, 'F': 165, 'G': 75, 'H': 155, 
         'I': 131, 'K': 146, 'L': 131, 'M': 149, 'N': 132, 'P': 115, 'Q': 146,
         'R': 174, 'S': 105, 'T': 119, 'V': 117, 'W': 204, 'Y': 181}

# Hydrophobicity scale (Kyte-Doolittle)
HYDRO_SCALE = {'A': 1.8, 'C': 2.5, 'D': -3.5, 'E': -3.5, 'F': 2.8, 'G': -0.4,
               'H': -3.2, 'I': 4.5, 'K': -3.9, 'L': 3.8, 'M': 1.9, 'N': -3.5,
               'P': -1.6, 'Q': -3.5, 'R': -4.5, 'S': -0.8, 'T': -0.7, 'V': 4.2,
               'W': -0.9, 'Y': -1.3}

def extract_features_enhanced(sequence):
    """Extract comprehensive features from protein sequence"""
    seq = ''.join(aa for aa in sequence.upper() if aa in AMINO_ACIDS)
    length = len(seq)
    
    if length == 0:
        return np.zeros(150)  # Updated feature size
    
    features = []
    
    # 1. Sequence length features (3)
    features.append(length / 10000)
    features.append(np.log1p(length) / 10)
    features.append(min(length, 1000) / 1000)  # Capped length
    
    # 2. Amino Acid Composition (20)
    aa_counts = Counter(seq)
    for aa in AMINO_ACIDS:
        features.append(aa_counts.get(aa, 0) / length)
    
    # 3. Physicochemical properties (15)
    features.append(sum(1 for aa in seq if aa in HYDROPHOBIC) / length)
    features.append(sum(1 for aa in seq if aa in HYDROPHILIC) / length)
    features.append(sum(1 for aa in seq if aa in POLAR) / length)
    features.append(sum(1 for aa in seq if aa in CHARGED_POS) / length)
    features.append(sum(1 for aa in seq if aa in CHARGED_NEG) / length)
    features.append(sum(1 for aa in seq if aa in AROMATIC) / length)
    features.append(sum(1 for aa in seq if aa in SMALL) / length)
    features.append(sum(1 for aa in seq if aa in ALIPHATIC) / length)
    features.append(sum(1 for aa in seq if aa in SULFUR) / length)
    features.append(sum(1 for aa in seq if aa in HYDROXYL) / length)
    
    # Net charge
    pos_charge = sum(1 for aa in seq if aa in CHARGED_POS)
    neg_charge = sum(1 for aa in seq if aa in CHARGED_NEG)
    features.append((pos_charge - neg_charge) / length)
    
    # Molecular weight
    mw = sum(AA_MW.get(aa, 110) for aa in seq)
    features.append(mw / (length * 200))  # Normalized
    
    # Hydrophobicity stats
    hydro_values = [HYDRO_SCALE.get(aa, 0) for aa in seq]
    features.append(np.mean(hydro_values) / 5)
    features.append(np.std(hydro_values) / 5)
    features.append(np.max(hydro_values) / 5 if hydro_values else 0)
    
    # 4. Sequence complexity (3)
    aa_freq = np.array([aa_counts.get(aa, 0) / length for aa in AMINO_ACIDS])
    aa_freq = aa_freq[aa_freq > 0]
    entropy = -np.sum(aa_freq * np.log2(aa_freq)) if len(aa_freq) > 0 else 0
    features.append(entropy / 4.5)
    
    # Unique AA ratio
    features.append(len(aa_counts) / 20)
    
    # Repeat ratio (consecutive same AA)
    repeats = sum(1 for i in range(len(seq)-1) if seq[i] == seq[i+1])
    features.append(repeats / max(1, length - 1))
    
    # 5. N-terminal and C-terminal signals (16)
    for term_seq, name in [(seq[:50], 'N'), (seq[-50:], 'C')]:
        term_len = len(term_seq)
        if term_len > 0:
            features.append(sum(1 for aa in term_seq if aa in HYDROPHOBIC) / term_len)
            features.append(sum(1 for aa in term_seq if aa in CHARGED_POS) / term_len)
            features.append(sum(1 for aa in term_seq if aa in CHARGED_NEG) / term_len)
            features.append(sum(1 for aa in term_seq if aa in POLAR) / term_len)
            features.append(sum(1 for aa in term_seq if aa in AROMATIC) / term_len)
            features.append(sum(1 for aa in term_seq if aa in SMALL) / term_len)
            # Hydrophobicity
            term_hydro = [HYDRO_SCALE.get(aa, 0) for aa in term_seq]
            features.append(np.mean(term_hydro) / 5)
            features.append(np.std(term_hydro) / 5 if len(term_hydro) > 1 else 0)
        else:
            features.extend([0] * 8)
    
    # 6. Dipeptide features (30 most informative)
    important_dipeptides = [
        'LL', 'AA', 'VV', 'GG', 'SS', 'EE', 'KK', 'AL', 'LA', 'EK',
        'KE', 'DE', 'ED', 'RK', 'LE', 'EL', 'AE', 'EA', 'LK', 'KL',
        'GS', 'SG', 'PP', 'PG', 'GP', 'VL', 'LV', 'IL', 'LI', 'TT'
    ]
    for dp in important_dipeptides:
        count = seq.count(dp)
        features.append(count / max(1, length - 1))
    
    # 7. Tripeptide features (20 key patterns)
    important_tripeptides = [
        'LLL', 'AAA', 'GGG', 'SSS', 'EEE', 'KKK', 'PPP', 'VVV',
        'LEK', 'EKL', 'ALA', 'GAS', 'VAL', 'ILE', 'PRO', 'GLY',
        'SER', 'THR', 'ASP', 'GLU'
    ]
    for tp in important_tripeptides:
        count = seq.count(tp)
        features.append(count / max(1, length - 2))
    
    # 8. Position-specific features (20)
    # Split sequence into 5 regions
    region_size = length // 5 if length >= 5 else length
    for i in range(5):
        start = i * region_size
        end = start + region_size if i < 4 else length
        region = seq[start:end]
        if len(region) > 0:
            features.append(sum(1 for aa in region if aa in HYDROPHOBIC) / len(region))
            features.append(sum(1 for aa in region if aa in CHARGED_POS) / len(region))
            features.append(sum(1 for aa in region if aa in CHARGED_NEG) / len(region))
            features.append(sum(1 for aa in region if aa in AROMATIC) / len(region))
        else:
            features.extend([0] * 4)
    
    # 9. Pseudo Amino Acid Composition (23)
    # Sequence-order correlation factors
    for lag in [1, 2, 3, 5, 10]:
        if length > lag:
            # Hydrophobicity correlation
            h_corr = sum(HYDRO_SCALE.get(seq[i], 0) * HYDRO_SCALE.get(seq[i+lag], 0) 
                        for i in range(length - lag)) / (length - lag)
            features.append(h_corr / 25)
        else:
            features.append(0)
    
    # Type I PseAAC features (18 more)
    for d in [1, 2, 3]:
        if length > d:
            hydro_diff = sum(abs(HYDRO_SCALE.get(seq[i], 0) - HYDRO_SCALE.get(seq[i+d], 0)) 
                            for i in range(length - d)) / (length - d)
            mw_diff = sum(abs(AA_MW.get(seq[i], 110) - AA_MW.get(seq[i+d], 110)) 
                         for i in range(length - d)) / (length - d)
            features.append(hydro_diff / 10)
            features.append(mw_diff / 200)
            
            # Charge transition
            charge_trans = sum(1 for i in range(length - d) 
                              if (seq[i] in CHARGED_POS) != (seq[i+d] in CHARGED_POS)) / (length - d)
            features.append(charge_trans)
            
            # Polarity transition
            polar_trans = sum(1 for i in range(length - d)
                             if (seq[i] in POLAR) != (seq[i+d] in POLAR)) / (length - d)
            features.append(polar_trans)
            
            # Size transition
            size_trans = sum(1 for i in range(length - d)
                            if (seq[i] in SMALL) != (seq[i+d] in SMALL)) / (length - d)
            features.append(size_trans)
            
            # Aromatic clustering
            arom_cluster = sum(1 for i in range(length - d)
                              if seq[i] in AROMATIC and seq[i+d] in AROMATIC) / (length - d)
            features.append(arom_cluster)
        else:
            features.extend([0] * 6)
    
    return np.array(features, dtype=np.float32)

# Test feature extraction
test_seq = list(train_sequences.values())[0]
test_features = extract_features_enhanced(test_seq)
print(f'\n🔧 Feature vector size: {len(test_features)}')
print(f'   Sample features (first 10): {test_features[:10]}')
print(f'   Non-zero features: {np.sum(test_features != 0)}')

⚙️ FEATURE ENGINEERING (ENHANCED)

🔧 Feature vector size: 150
   Sample features (first 10): [0.0016     0.28332132 0.016      0.         0.         0.
 0.0625     0.0625     0.0625     0.        ]
   Non-zero features: 83


In [ ]:
# Extract features for all training proteins
print('\n🧬 Extracting features for training proteins...')

proteins = train_terms['protein_id'].unique()
all_go_terms = train_terms['go_term'].unique()

print(f'   Proteins in train_terms: {len(proteins):,}')
print(f'   Sequences available: {len(train_sequences):,}')

# Debug: Check matching
matched = sum(1 for p in proteins if p in train_sequences)
print(f'   Proteins with sequences: {matched:,}')

if matched == 0:
    print('\n❌ ERROR: No proteins match! Checking IDs...')
    print(f'   First 3 train_terms IDs: {proteins[:3].tolist()}')
    print(f'   First 3 sequence IDs: {list(train_sequences.keys())[:3]}')
    raise ValueError("No matching protein IDs between train_terms and train_sequences!")

# Extract features with ENHANCED function
X_features = []
valid_proteins = []

for pid in tqdm(proteins, desc='Extracting features'):
    seq = train_sequences.get(pid, '')
    if seq:
        X_features.append(extract_features_enhanced(seq))
        valid_proteins.append(pid)

X = np.array(X_features)
print(f'\n✅ Features extracted: {X.shape}')
print(f'   Valid proteins: {len(valid_proteins):,} / {len(proteins):,}')

if len(valid_proteins) == 0:
    raise ValueError("No features extracted! Check protein ID matching.")

# 📊 Tạo label matrix - VECTORIZED
print('\n📊 Creating label matrix (vectorized)...')

protein_to_idx = {p: i for i, p in enumerate(valid_proteins)}
term_to_idx = {t: i for i, t in enumerate(all_go_terms)}

# Vectorized approach - much faster than iterrows
valid_mask = train_terms['protein_id'].isin(protein_to_idx)
valid_terms_df = train_terms[valid_mask]

row_indices = valid_terms_df['protein_id'].map(protein_to_idx).values
col_indices = valid_terms_df['go_term'].map(term_to_idx).values

y = np.zeros((len(valid_proteins), len(all_go_terms)), dtype=np.int8)
y[row_indices, col_indices] = 1

print(f'✅ Label matrix: {y.shape}')
print(f'   Positive labels: {y.sum():,} ({100*y.sum()/(y.shape[0]*y.shape[1]):.4f}%)')


🧬 Extracting features for training proteins...
   Proteins in train_terms: 82,404
   Sequences available: 82,404
   Proteins with sequences: 82,404


Extracting features: 100%|██████████| 82404/82404 [03:34<00:00, 383.61it/s]



✅ Features extracted: (82404, 150)
   Valid proteins: 82,404 / 82,404

📊 Creating label matrix (vectorized)...
✅ Label matrix: (82404, 26125)
   Positive labels: 537,027 (0.0249%)


In [ ]:
# Scale Features & Filter GO Terms & Create IA Weights
print('\n📐 Scaling features...')
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print(f'✅ Features scaled: {X_scaled.shape}')

# Filter GO terms
MIN_PROTEINS_PER_TERM = 20 

term_counts = y.sum(axis=0)
valid_mask = term_counts >= MIN_PROTEINS_PER_TERM

print(f'\n📊 GO Term Filtering:')
print(f'   • Total: {len(term_counts):,}')
print(f'   • With ≥{MIN_PROTEINS_PER_TERM} proteins: {valid_mask.sum():,}')

y_filtered = y[:, valid_mask]
filtered_go_terms = all_go_terms[valid_mask]

print(f'✅ Filtered: {y_filtered.shape[1]:,} GO terms')

# ⚖️ Prepare IA Weights for Loss Function
print('\n⚖️ Preparing IA weights for weighted loss...')

if ia_weights is not None:
    ia_dict = dict(zip(ia_weights['go_term'], ia_weights['ia_weight']))
    term_ia_weights = np.array([ia_dict.get(t, 1.0) for t in filtered_go_terms])
    # Normalize weights
    term_ia_weights = term_ia_weights / term_ia_weights.mean()
    print(f'   IA weights range: [{term_ia_weights.min():.3f}, {term_ia_weights.max():.3f}]')
else:
    term_ia_weights = np.ones(len(filtered_go_terms))
    print('   Using uniform weights (IA.tsv not found)')

#  Calculate Class Weights for Imbalanced Data
print('\n⚖️ Calculating class weights...')
pos_counts = y_filtered.sum(axis=0)
neg_counts = y_filtered.shape[0] - pos_counts

# Effective number of samples weighting
beta = 0.9999
pos_weights = (1 - beta) / (1 - np.power(beta, pos_counts + 1))
neg_weights = (1 - beta) / (1 - np.power(beta, neg_counts + 1))

# Combine with IA weights
class_weights = (neg_weights / pos_weights) * term_ia_weights
class_weights = np.clip(class_weights, 0.5, 50)  # Clip extreme values
print(f'   Class weights range: [{class_weights.min():.2f}, {class_weights.max():.2f}]')


📐 Scaling features...
✅ Features scaled: (82404, 150)

📊 GO Term Filtering:
   • Total: 26,125
   • With ≥20 proteins: 4,227
✅ Filtered: 4,227 GO terms

⚖️ Preparing IA weights for weighted loss...
   IA weights range: [0.000, 7.435]

⚖️ Calculating class weights...
   Class weights range: [0.50, 0.67]


## 🤖 BƯỚC 5: Training Neural Network

Sử dụng Neural Network với Residual Blocks cho multi-label classification.

In [ ]:
print('='*60)
print('🤖 MODEL TRAINING SETUP (MEMORY OPTIMIZED)')
print('='*60)

import gc

# CẤU HÌNH
MAX_GO_TERMS = 1500      
HIDDEN_DIM = 512       
BATCH_SIZE = 256         
NUM_EPOCHS = 25          
LEARNING_RATE = 5e-4     
PROB_THRESHOLD = 0.01    

print(f'\n⚙️ Configuration (Memory Optimized):')
print(f'   • Max GO terms: {MAX_GO_TERMS}')
print(f'   • Hidden dim: {HIDDEN_DIM}')
print(f'   • Epochs: {NUM_EPOCHS}')
print(f'   • Batch size: {BATCH_SIZE}')

# Clear unused memory
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Train-validation split
print('\n📊 Splitting data...')
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y_filtered,
    test_size=0.1,
    random_state=42
)
print(f'   Train: {X_train.shape[0]:,} samples')
print(f'   Val: {X_val.shape[0]:,} samples')

# Select top GO terms
term_counts = y_train.sum(axis=0)
top_term_indices = np.argsort(term_counts)[-MAX_GO_TERMS:]
y_train = y_train[:, top_term_indices]
y_val = y_val[:, top_term_indices]
filtered_go_terms_final = filtered_go_terms[top_term_indices]
final_class_weights = class_weights[top_term_indices]

# Free memory from filtered arrays
del y_filtered
gc.collect()

print(f'   GO terms: {y_train.shape[1]:,}')
print(f'   Positive labels in train: {y_train.sum():,}')

🤖 MODEL TRAINING SETUP (MEMORY OPTIMIZED)

⚙️ Configuration (Memory Optimized):
   • Max GO terms: 1500
   • Hidden dim: 512
   • Epochs: 25
   • Batch size: 256

📊 Splitting data...
   Train: 74,163 samples
   Val: 8,241 samples
   GO terms: 1,500
   Positive labels in train: 308,091


In [ ]:
# Neural Network Model (IMPROVED)

class SEBlock(nn.Module):
    """Squeeze-and-Excitation block for channel attention"""
    def __init__(self, dim, reduction=16):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(dim, dim // reduction),
            nn.ReLU(),
            nn.Linear(dim // reduction, dim),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        scale = self.fc(x)
        return x * scale

class ResidualBlock(nn.Module):
    def __init__(self, dim, dropout=0.2):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(dim, dim),
            nn.BatchNorm1d(dim),
            nn.GELU(), 
            nn.Dropout(dropout),
            nn.Linear(dim, dim),
            nn.BatchNorm1d(dim)
        )
        self.se = SEBlock(dim)
        self.act = nn.GELU()
        
    def forward(self, x):
        residual = x
        out = self.block(x)
        out = self.se(out)
        return self.act(out + residual)

class ProteinClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super().__init__()
        
        # Input projection with wider first layer
        self.input_proj = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.GELU(),
            nn.Dropout(0.3),
        )
        
        # Residual blocks
        self.res_blocks = nn.Sequential(
            ResidualBlock(hidden_dim, 0.25),
            ResidualBlock(hidden_dim, 0.25),
            ResidualBlock(hidden_dim, 0.25),
            ResidualBlock(hidden_dim, 0.20),
        )
        
        # Output layers
        self.output = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim // 2, num_classes)
        )
        
    def forward(self, x):
        x = self.input_proj(x)
        x = self.res_blocks(x)
        return self.output(x)

# 📦 Focal Loss for Imbalanced Multi-Label Classification
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, pos_weight=None):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.pos_weight = pos_weight
        
    def forward(self, inputs, targets):
        BCE_loss = nn.functional.binary_cross_entropy_with_logits(
            inputs, targets, reduction='none', pos_weight=self.pos_weight
        )
        pt = torch.exp(-BCE_loss)
        focal_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        return focal_loss.mean()

# 📦 Prepare DataLoaders
train_dataset = TensorDataset(
    torch.FloatTensor(X_train), 
    torch.FloatTensor(y_train)
)
val_dataset = TensorDataset(
    torch.FloatTensor(X_val), 
    torch.FloatTensor(y_val)
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE * 2, shuffle=False, num_workers=0, pin_memory=True)

# 🏋️ Training with Improved Setup
print('\n🏋️ Training Neural Network (Improved)...')
print(f'   Using device: {device}')

model = ProteinClassifier(X_train.shape[1], HIDDEN_DIM, y_train.shape[1]).to(device)
print(f'   Model parameters: {sum(p.numel() for p in model.parameters()):,}')

# Optimizer with gradient clipping
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.02)

# Cosine annealing with warm restarts
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)

# Focal Loss with class weights
pos_weight_tensor = torch.FloatTensor(final_class_weights).to(device)
criterion = FocalLoss(alpha=1.0, gamma=2.0, pos_weight=pos_weight_tensor)

start_time = time.time()
best_val_loss = float('inf')
patience = 7
patience_counter = 0

for epoch in range(NUM_EPOCHS):
    # Train
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        train_loss += loss.item()
    
    scheduler.step()
    
    # Validate
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            val_loss += criterion(outputs, y_batch).item()
    
    avg_train = train_loss / len(train_loader)
    avg_val = val_loss / len(val_loader)
    current_lr = optimizer.param_groups[0]['lr']
    
    # Early stopping check
    if avg_val < best_val_loss:
        best_val_loss = avg_val
        best_model_state = model.state_dict().copy()
        patience_counter = 0
    else:
        patience_counter += 1
    
    if (epoch + 1) % 5 == 0 or epoch == NUM_EPOCHS - 1:
        print(f'   Epoch {epoch+1:2d}/{NUM_EPOCHS} | Train: {avg_train:.4f} | Val: {avg_val:.4f} | LR: {current_lr:.6f}')
    
    if patience_counter >= patience:
        print(f'\n   ⚠️ Early stopping at epoch {epoch+1}')
        break

# Load best model
model.load_state_dict(best_model_state)
training_time = time.time() - start_time

print(f'\n✅ Training completed in {training_time:.1f}s ({training_time/60:.1f} min)')
print(f'   Best Val Loss: {best_val_loss:.4f}')


🏋️ Training Neural Network (Improved)...
   Using device: cuda
   Model parameters: 2,838,364
   Epoch  5/25 | Train: 0.0018 | Val: 0.0018 | LR: 0.000250
   Epoch 10/25 | Train: 0.0018 | Val: 0.0017 | LR: 0.000500
   Epoch 15/25 | Train: 0.0016 | Val: 0.0016 | LR: 0.000427
   Epoch 20/25 | Train: 0.0015 | Val: 0.0015 | LR: 0.000250
   Epoch 25/25 | Train: 0.0014 | Val: 0.0015 | LR: 0.000073

✅ Training completed in 140.9s (2.3 min)
   Best Val Loss: 0.0015


In [ ]:
# Evaluate Model & Find Optimal Threshold
print('\n📈 Evaluating model...')

model.eval()

def predict_batch(X):
    """Predict probabilities with model"""
    X_tensor = torch.FloatTensor(X).to(device)
    with torch.no_grad():
        outputs = model(X_tensor)
        probs = torch.sigmoid(outputs).cpu().numpy()
    return probs

# Get predictions
val_probs = predict_batch(X_val)
train_probs = predict_batch(X_train)

# Find Optimal Threshold using F1
print('\n🎯 Finding optimal threshold...')

from sklearn.metrics import f1_score

best_f1 = 0
best_threshold = 0.01

for threshold in [0.005, 0.01, 0.02, 0.03, 0.05, 0.07, 0.1]:
    val_preds = (val_probs >= threshold).astype(np.int8)
    
    # Skip if no predictions
    if val_preds.sum() == 0:
        continue
    
    # Micro F1 (what competition uses)
    f1_micro = f1_score(y_val, val_preds, average='micro', zero_division=0)
    
    # Coverage (how many GO terms predicted)
    coverage = val_preds.sum(axis=1).mean()
    
    print(f'   Threshold {threshold:.3f}: F1={f1_micro:.4f}, Avg predictions={coverage:.1f}')
    
    if f1_micro > best_f1:
        best_f1 = f1_micro
        best_threshold = threshold

print(f'\n✅ Best threshold: {best_threshold} (F1={best_f1:.4f})')
PROB_THRESHOLD = best_threshold

# Final evaluation with best threshold
val_preds = (val_probs >= PROB_THRESHOLD).astype(np.int8)
train_preds = (train_probs >= PROB_THRESHOLD).astype(np.int8)

val_hamming = hamming_loss(y_val, val_preds)
train_hamming = hamming_loss(y_train, train_preds)

print(f'\n📊 Final Results (threshold={PROB_THRESHOLD}):')
print(f'   Train Hamming Loss: {train_hamming:.6f}')
print(f'   Val Hamming Loss: {val_hamming:.6f}')
print(f'   Val F1 (micro): {best_f1:.4f}')
print(f'   Val Predictions: {val_preds.sum():,} positives ({100*val_preds.mean():.2f}%)')
print(f'   Unique GO terms predicted: {(val_preds.sum(axis=0) > 0).sum():,}')


📈 Evaluating model...

🎯 Finding optimal threshold...
   Threshold 0.005: F1=0.0056, Avg predictions=1475.6
   Threshold 0.010: F1=0.0059, Avg predictions=1384.7
   Threshold 0.020: F1=0.0073, Avg predictions=1108.8
   Threshold 0.030: F1=0.0096, Avg predictions=835.6
   Threshold 0.050: F1=0.0174, Avg predictions=429.9
   Threshold 0.070: F1=0.0319, Avg predictions=209.3
   Threshold 0.100: F1=0.0712, Avg predictions=74.9

✅ Best threshold: 0.1 (F1=0.0712)

📊 Final Results (threshold=0.1):
   Train Hamming Loss: 0.048988
   Val Hamming Loss: 0.048920
   Val F1 (micro): 0.0712
   Val Predictions: 617,214 positives (4.99%)
   Unique GO terms predicted: 1,500


## 🎯 BƯỚC 6: Generate Test Predictions

In [16]:
print('='*60)
print('🎯 GENERATING TEST PREDICTIONS')
print('='*60)

# Extract features for test proteins with ENHANCED function
print('\n🧬 Extracting test features (enhanced)...')

test_protein_ids = list(test_sequences.keys())
X_test_features = []

for pid in tqdm(test_protein_ids, desc='Extracting test features'):
    X_test_features.append(extract_features_enhanced(test_sequences[pid]))

X_test = np.array(X_test_features)
X_test_scaled = scaler.transform(X_test)

print(f'✅ Test features: {X_test_scaled.shape}')

🎯 GENERATING TEST PREDICTIONS

🧬 Extracting test features (enhanced)...


Extracting test features: 100%|██████████| 224309/224309 [08:00<00:00, 466.41it/s] 


✅ Test features: (224309, 150)


In [ ]:
#  Make Predictions (Batched)
print('\n🚀 Making predictions...')

model.eval()
batch_size = 10000
all_probs = []

for i in tqdm(range(0, len(X_test_scaled), batch_size), desc='Predicting'):
    batch_X = X_test_scaled[i:i+batch_size]
    X_tensor = torch.FloatTensor(batch_X).to(device)
    
    with torch.no_grad():
        outputs = model(X_tensor)
        probs = torch.sigmoid(outputs).cpu().numpy()
    all_probs.append(probs)

y_test_proba = np.vstack(all_probs)
print(f'✅ Predictions shape: {y_test_proba.shape}')


🚀 Making predictions...


Predicting: 100%|██████████| 23/23 [00:01<00:00, 14.06it/s]


✅ Predictions shape: (224309, 1500)


## 📝 BƯỚC 7: Create Submission File

In [ ]:
# CREATE SUBMISSION WITH GO HIERARCHY PROPAGATION
print('='*60)
print('📝 CREATING SUBMISSION FILE')
print('='*60)

import os
import shutil

submission_file = SUBMISSION_PATH / 'submission.tsv'
propagated_file = SUBMISSION_PATH / 'submission_propagated.tsv'

for f in [submission_file, propagated_file]:
    if f.exists():
        os.remove(f)
        print(f'   🗑️ Đã xóa file cũ: {f}')

# Root terms that are too general
ROOT_TERMS = {
    'GO:0008150',  # biological_process
    'GO:0003674',  # molecular_function  
    'GO:0005575',  # cellular_component
    'GO:0005623',  # cell
    'GO:0044464',  # cell part
    'GO:0044424',  # intracellular part
    'GO:0005622',  # intracellular
}

# Very common terms to down-weight
COMMON_GENERAL_TERMS = {
    'GO:0005886',  # plasma membrane
    'GO:0005737',  # cytoplasm
    'GO:0005634',  # nucleus
    'GO:0016020',  # membrane
    'GO:0005829',  # cytosol
    'GO:0005515',  # protein binding
}

#  GO HIERARCHY PROPAGATION
if GO_HIERARCHY_LOADED:
    print('\n🌳 Applying GO Hierarchy Propagation...')
    
    # Build IA dictionary from ia_weights DataFrame
    if ia_weights is not None:
        ia_dict = dict(zip(ia_weights['go_term'], ia_weights['ia_weight']))
        max_ia = max(ia_dict.values()) if ia_dict else 1.0
    else:
        ia_dict = {}
        max_ia = 1.0
    
    # Build filtered ancestors (exclude root terms)
    print('   Pre-computing mappings...')
    term_ancestors_filtered = {}
    for term in filtered_go_terms_final:
        ancestors = ancestor_cache.get(term, {term})
        filtered_ancestors = ancestors - ROOT_TERMS
        term_ancestors_filtered[term] = filtered_ancestors
    
    print(f'   Model GO terms: {len(filtered_go_terms_final):,}')
    
    # Precompute weights for common terms
    all_terms_in_ancestors = set()
    for ancestors in term_ancestors_filtered.values():
        all_terms_in_ancestors.update(ancestors)
    
    term_weights = {t: 0.7 if t in COMMON_GENERAL_TERMS else 1.0 
                   for t in all_terms_in_ancestors}
    
    # Precompute IA weights
    if ia_dict:
        term_ia_final = {t: ia_dict.get(t, 1.0) / max_ia 
                        for t in all_terms_in_ancestors}
    else:
        term_ia_final = None
    
    PROPAGATION_THRESHOLD = 0.03
    MAX_PREDS_PROPAGATED = 100
    
    print(f'   Threshold: {PROPAGATION_THRESHOLD}, Max per protein: {MAX_PREDS_PROPAGATED}')
    print(f'   Processing {len(test_protein_ids):,} proteins...\n')
    
    total_predictions = 0
    go_term_counts = {}
    
    # Process all proteins and write directly
    with open(propagated_file, 'w') as f:
        for i in tqdm(range(len(test_protein_ids)), desc='Creating submission', mininterval=0.5):
            protein_id = test_protein_ids[i]
            probs = y_test_proba[i]
            
            # Find top predictions
            top_indices = np.argsort(-probs)[:20]
            
            # Filter by threshold or keep top 5
            mask = probs[top_indices] >= PROPAGATION_THRESHOLD
            if mask.sum() > 0:
                top_indices = top_indices[mask]
            else:
                top_indices = top_indices[:5]
            
            # Propagate to ancestors
            term_scores = {}
            for idx in top_indices:
                term = filtered_go_terms_final[idx]
                prob = float(probs[idx])
                
                # Add term and ancestors
                for ancestor in term_ancestors_filtered.get(term, {term}):
                    weight = term_weights.get(ancestor, 1.0)
                    weighted_prob = prob * weight
                    
                    if ancestor not in term_scores or weighted_prob > term_scores[ancestor]:
                        term_scores[ancestor] = weighted_prob
            
            # Get top predictions
            sorted_items = sorted(term_scores.items(), key=lambda x: -x[1])[:MAX_PREDS_PROPAGATED]
            
            # Write lines
            for term, prob in sorted_items:
                # Apply IA weight
                if term_ia_final:
                    ia_w = term_ia_final.get(term, 1.0)
                    final_prob = prob * (0.7 + 0.3 * ia_w)
                else:
                    final_prob = prob
                
                f.write(f'{protein_id}\t{term}\t{final_prob:.6f}\n')
                total_predictions += 1
                go_term_counts[term] = go_term_counts.get(term, 0) + 1
    
    print(f'\n📊 Results:')
    print(f'   Total predictions: {total_predictions:,}')
    print(f'   Unique GO terms: {len(go_term_counts):,}')
    print(f'   Avg per protein: {total_predictions / len(test_protein_ids):.1f}')
    
    # Show top GO terms
    print(f'\n🔍 Top 10 GO terms:')
    for i, (term, count) in enumerate(sorted(go_term_counts.items(), key=lambda x: -x[1])[:10], 1):
        name = go_names.get(term, '')[:35]
        pct = 100 * count / len(test_protein_ids)
        print(f'   {i}. {term}: {count:,} ({pct:.1f}%) - {name}')
    
    # Copy to final submission
    shutil.copy(propagated_file, submission_file)
    
else:
    # No GO hierarchy - create simple submission
    print('\n⚠️ GO hierarchy not loaded - creating simple submission...')
    
    SIMPLE_THRESHOLD = 0.03
    MAX_PREDS = 100
    total_predictions = 0
    
    with open(submission_file, 'w') as f:
        for i in tqdm(range(len(test_protein_ids)), desc='Creating submission'):
            protein_id = test_protein_ids[i]
            probs = y_test_proba[i]
            
            top_indices = np.argsort(-probs)[:MAX_PREDS]
            mask = probs[top_indices] >= SIMPLE_THRESHOLD
            if mask.sum() > 0:
                top_indices = top_indices[mask]
            else:
                top_indices = top_indices[:5]
            
            for idx in top_indices:
                term = filtered_go_terms_final[idx]
                prob = float(probs[idx])
                f.write(f'{protein_id}\t{term}\t{prob:.6f}\n')
                total_predictions += 1
    
    print(f'   Total predictions: {total_predictions:,}')

# Verify file
file_size = os.path.getsize(submission_file) / (1024 * 1024)
print(f'\n✅ Submission saved: {submission_file}')
print(f'   Size: {file_size:.2f} MB')

📝 CREATING SUBMISSION FILE

🌳 Applying GO Hierarchy Propagation...
   Pre-computing mappings...
   Model GO terms: 1,500
   Threshold: 0.03, Max per protein: 100
   Processing 224,309 proteins...



Creating submission: 100%|██████████| 224309/224309 [00:43<00:00, 5182.39it/s]



📊 Results:
   Total predictions: 15,724,114
   Unique GO terms: 2,558
   Avg per protein: 70.1

🔍 Top 10 GO terms:
   1. GO:0110165: 224,309 (100.0%) - cellular anatomical structure
   2. GO:0005488: 224,309 (100.0%) - binding
   3. GO:0005515: 224,230 (100.0%) - protein binding
   4. GO:0005737: 221,477 (98.7%) - cytoplasm
   5. GO:0043226: 217,812 (97.1%) - organelle
   6. GO:0043227: 217,385 (96.9%) - membrane-bounded organelle
   7. GO:0043229: 214,928 (95.8%) - intracellular organelle
   8. GO:0043231: 214,834 (95.8%) - intracellular membrane-bounded orga
   9. GO:0005829: 213,551 (95.2%) - cytosol
   10. GO:0016020: 194,891 (86.9%) - membrane

✅ Submission saved: /kaggle/working/submission.tsv
   Size: 406.00 MB


In [19]:
# Preview submission
print('\n📄 Submission Preview:')
preview_df = pd.read_csv(submission_file, sep='\t', header=None, names=['protein_id', 'go_term', 'confidence'], nrows=10)
print(preview_df.to_string(index=False))

print('\n🏆 Top GO Terms in submission:')
# Read just go_term column for stats
go_counts = pd.read_csv(submission_file, sep='\t', header=None, usecols=[1], names=['go_term'])['go_term'].value_counts().head(10)
for i, (term, count) in enumerate(go_counts.items(), 1):
    name = go_names.get(term, 'Unknown') if GO_HIERARCHY_LOADED else ''
    print(f'   {i}. {term}: {count:,} ({name[:40]}...)')


📄 Submission Preview:
protein_id    go_term  confidence
A0A0C5B5G6 GO:0043231    0.267249
A0A0C5B5G6 GO:0043227    0.267981
A0A0C5B5G6 GO:0043226    0.270981
A0A0C5B5G6 GO:0005739    0.281662
A0A0C5B5G6 GO:0043229    0.267155
A0A0C5B5G6 GO:0110165    0.267185
A0A0C5B5G6 GO:0005743    0.235178
A0A0C5B5G6 GO:0005740    0.235892
A0A0C5B5G6 GO:0031090    0.239600
A0A0C5B5G6 GO:0031966    0.235190

🏆 Top GO Terms in submission:
   1. GO:0005488: 224,309 (binding...)
   2. GO:0110165: 224,309 (cellular anatomical structure...)
   3. GO:0005515: 224,230 (protein binding...)
   4. GO:0005737: 221,477 (cytoplasm...)
   5. GO:0043226: 217,812 (organelle...)
   6. GO:0043227: 217,385 (membrane-bounded organelle...)
   7. GO:0043229: 214,928 (intracellular organelle...)
   8. GO:0043231: 214,834 (intracellular membrane-bounded organelle...)
   9. GO:0005829: 213,551 (cytosol...)
   10. GO:0016020: 194,891 (membrane...)


## ✅ BƯỚC 8: Summary

In [20]:
print('\n' + '='*60)
print('🎉 PIPELINE COMPLETE!')
print('='*60)

# Get stats from file
import os
file_size = os.path.getsize(submission_file) / (1024 * 1024)  # MB

print(f'''
📊 SUMMARY:
{'='*50}

📁 Data:
   • Training proteins: {len(valid_proteins):,}
   • Test proteins: {len(test_protein_ids):,}
   • GO terms trained: {len(filtered_go_terms_final):,}

🧬 Features: {X_scaled.shape[1]} dimensions
   • AAC, Physicochemical, Dipeptides, PseAAC

🤖 Model: Neural Network
   • Hidden: {HIDDEN_DIM}, ResBlocks: 4
   • Focal Loss + IA Weights
   • Training time: {training_time/60:.1f} min

🌳 GO Propagation: {"✅ ENABLED" if GO_HIERARCHY_LOADED else "❌"}

📝 Submission:
   • File: {submission_file}
   • Size: {file_size:.1f} MB

{'='*50}
✅ Ready for Kaggle submission!
''')


🎉 PIPELINE COMPLETE!

📊 SUMMARY:

📁 Data:
   • Training proteins: 82,404
   • Test proteins: 224,309
   • GO terms trained: 1,500

🧬 Features: 150 dimensions
   • AAC, Physicochemical, Dipeptides, PseAAC

🤖 Model: Neural Network
   • Hidden: 512, ResBlocks: 4
   • Focal Loss + IA Weights
   • Training time: 2.3 min

🌳 GO Propagation: ✅ ENABLED

📝 Submission:
   • File: /kaggle/working/submission.tsv
   • Size: 406.0 MB

✅ Ready for Kaggle submission!



In [21]:
from IPython.display import FileLink
FileLink('submission.tsv')

/kaggle/working/submission.tsv